In [1]:
#importing spark and creating spark sessions
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("streaming").master("local[*]").getOrCreate()

/home/ec2-user/spark-3.2.0-bin-hadoop3.2/python/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [2]:
#importing required libraries
import math
from IPython.display import clear_output, display
from time import sleep
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import Row
import datetime
from datetime import datetime, timedelta
from functools import reduce

#importing HTML for better display
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
#Creating the schema for data to be received via spark streaming
dataSchema = StructType([StructField("medallion",StringType(),True), 
           StructField("hack_license",StringType(),True), 
           StructField("pickup_datetime",TimestampType(),True), 
           StructField("dropoff_datetime",TimestampType(),True), 
           StructField("trip_time_in_secs",StringType(),True), 
           StructField("trip_distance",StringType(),True), 
           StructField("pickup_longitude",FloatType(),True), 
           StructField("pickup_latitude",FloatType(),True), 
           StructField("dropoff_longitude",FloatType(),True), 
           StructField("dropoff_latitude",FloatType(),True), 
           StructField("payment_type",StringType(),True), 
           StructField("fare_amount",FloatType(),True), 
           StructField("surcharge",FloatType(),True), 
           StructField("mta_tax",FloatType(),True), 
           StructField("tip_amount",FloatType(),True), 
           StructField("tolls_amount",FloatType(),True), 
           StructField("total_amount",FloatType(),True)
                       ])

In [4]:
#Reading the csv data into the dataframe
df = (spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).csv("sampled_data", timestampFormat= "dd-MM-yyyy HH:mm"))

In [5]:
#Giving the specifications of latitudes and longitudes
start_lat = 41.474937      #latitude for starting cell
start_long = -74.913585    #longitude for starting cell
lat_degree = 0.004491556   #Change in degrees for latitude
long_degree = 0.004491556  #Change in degrees for longitude
Grid_size = 150            #Grid size for finding most frequent routes in kms
cell_size = 0.5            #Cell size for finding most frequent routes in kms

In [6]:
#Function defined for calculating grid cells
def get_grids(longitude, latitude):
    if longitude == None or latitude == None:              #Removing distorted data
        return None
    long = int(abs(longitude - start_long)/long_degree)    #Generating first half of grid cell
    lat = int(abs(latitude - start_lat)/lat_degree)        #Generating second half of grid cell
    if (long <= 300 and long >= 0 and lat <= 300 and lat >= 0): 
        result = (str(long) + "." + str(lat))              # Getting the grid cells in proper format
        return float(result)

In [7]:
#Creating a UDF for pyspark streaming dataframe
grid = udf(get_grids, FloatType())
df = df.withColumn("start_cell", grid(df.pickup_longitude, df.pickup_latitude))\
       .withColumn("end_cell", grid(df.dropoff_longitude, df.dropoff_latitude))
df = df.where(df.start_cell != df.end_cell)  #Removing the discrepency of data

In [11]:
from pyspark.sql.functions import *

#Creating the function to run on each batch
def batch_operation(df, epoch_id):
    start = datetime.now()      #Start time for analysing the delay
    
     #Creating the window of 30 min and removing nulls
    df = df.groupBy(window("dropoff_datetime", "30 minutes"), "start_cell","end_cell").count().dropna()  
    df = df.orderBy(df["window"], ascending = False)  #Ordering data by window in descending order
    df = df.orderBy(df["count"], ascending = False)    #Ordering data by count in descending order
    
    windowDf = Window.partitionBy(df["window"]).orderBy(col("window").desc()) #Partitioning window
    df = df.withColumn("cells", row_number().over(windowDf))                  #adding the cell ids column
    
    df = df.filter(col("cells") <= 10)                                        #filtering the data by 10
    
    df = df.withColumn("window", col("window").cast(StringType()))            #changing the type of window column
    df = df.withColumn("pickup_datetime", split(col("window"), ",").getItem(0))\
          .withColumn("dropoff_datetime", split(col("window"), ",").getItem(1)).drop("window")
    df = df.withColumn("pickup_datetime",expr("substring(pickup_datetime, 2, 20)"))\
           .withColumn("dropoff_datetime",expr("substring(dropoff_datetime, 2, 20)")) #removing discrepencies of data
        
    end = datetime.now()             #End time for analysing the delay
    delay = end - start              #generating the delay
    
    df = df.withColumn("delay(in seconds)", lit(str(delay.total_seconds()))) #adding delay column for each data set
    
    #selecting the required columns in a particular format
    df = df.select("pickup_datetime", "dropoff_datetime","cells", "start_cell","end_cell", "count", "delay(in seconds)" )
    
    #writing the data in csv format into output folder
    df.repartition(1).write.mode("append").csv('/home/ec2-user/output_folder_query1/', header = True)
    df.show(truncate = False)
   

In [12]:
#querying the data for batch processing
query = (df.writeStream.format("memory")\
           .foreachBatch(batch_operation)\
           .queryName("query1")\
           .outputMode("update")\
           .option("truncate",False)\
           .start())
df.isStreaming

True

In [13]:
#displaying the streaming data for 5 minutes
end_time = datetime.now() + timedelta(minutes=5)
df.createOrReplaceTempView("query1")      #replacing the view if already created
while True:                               #runs for displaying streaming data
    current_time = datetime.now()
    if current_time > end_time:           #if the time limit is exceeded then stops the loop
        break
    display(query.status)
    df_final = spark.sql('SELECT * FROM query1') #using spark sql to display the data
    display(df_final)                            #displaying the final data
    sleep(3)                                     #adding a sleep time of 3 seconds

{'isDataAvailable': True,
 'isTriggerActive': True,
 'message': 'Processing new data'}

DataFrame[medallion: string, hack_license: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, trip_time_in_secs: string, trip_distance: string, pickup_longitude: float, pickup_latitude: float, dropoff_longitude: float, dropoff_latitude: float, payment_type: string, fare_amount: float, surcharge: float, mta_tax: float, tip_amount: float, tolls_amount: float, total_amount: float, start_cell: float, end_cell: float]

{'isDataAvailable': True,
 'isTriggerActive': True,
 'message': 'Processing new data'}

DataFrame[medallion: string, hack_license: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, trip_time_in_secs: string, trip_distance: string, pickup_longitude: float, pickup_latitude: float, dropoff_longitude: float, dropoff_latitude: float, payment_type: string, fare_amount: float, surcharge: float, mta_tax: float, tip_amount: float, tolls_amount: float, total_amount: float, start_cell: float, end_cell: float]

{'isDataAvailable': True,
 'isTriggerActive': True,
 'message': 'Processing new data'}

...

{'isDataAvailable': True,
 'isTriggerActive': True,
 'message': 'Processing new data'}

DataFrame[medallion: string, hack_license: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, trip_time_in_secs: string, trip_distance: string, pickup_longitude: float, pickup_latitude: float, dropoff_longitude: float, dropoff_latitude: float, payment_type: string, fare_amount: float, surcharge: float, mta_tax: float, tip_amount: float, tolls_amount: float, total_amount: float, start_cell: float, end_cell: float]

{'isDataAvailable': True,
 'isTriggerActive': True,
 'message': 'Processing new data'}

DataFrame[medallion: string, hack_license: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, trip_time_in_secs: string, trip_distance: string, pickup_longitude: float, pickup_latitude: float, dropoff_longitude: float, dropoff_latitude: float, payment_type: string, fare_amount: float, surcharge: float, mta_tax: float, tip_amount: float, tolls_amount: float, total_amount: float, start_cell: float, end_cell: float]

+-------------------+--------------------+-----+----------+--------+-----+-----------------+
|pickup_datetime    |dropoff_datetime    |cells|start_cell|end_cell|count|delay(in seconds)|
+-------------------+--------------------+-----+----------+--------+-----+-----------------+
|2013-02-03 06:30:00|2013-02-03 07:00:00}|1    |213.156   |212.155 |2    |0.174929         |
|2013-02-03 06:30:00|2013-02-03 07:00:00}|2    |205.16    |206.162 |2    |0.174929         |
|2013-02-03 06:30:00|2013-02-03 07:00:00}|3    |204.163   |205.16  |2    |0.174929         |
|2013-02-03 06:30:00|2013-02-03 07:00:00}|4    |212.158   |206.159 |2    |0.174929         |
|2013-02-03 06:30:00|2013-02-03 07:00:00}|5    |213.155   |210.159 |2    |0.174929         |
|2013-02-03 06:30:00|2013-02-03 07:00:00}|6    |210.16    |204.158 |2    |0.174929         |
|2013-02-03 06:30:00|2013-02-03 07:00:00}|7    |205.161   |205.159 |2    |0.174929         |
|2013-02-03 06:30:00|2013-02-03 07:00:00}|8    |221.162   |215.155 |1 